# Enrichissement Sémantique d'un Corpus Visuel

**Contexte :** Ce notebook présente un protocole de traitement pour l'analyse sémantique d'un corpus d'images numériques. L'objectif est d'appliquer des méthodes d'intelligence artificielle pour extraire automatiquement des mots-clés (tags) descriptifs, transformant ainsi une collection d'images brutes en une base de données structurée et interrogeable.

**Démarche :** La méthodologie employée ici s'inscrit pleinement dans les pratiques des **Humanités Numériques**. Elle vise à combiner la rigueur technique avec une réflexion critique sur les outils, en vue de répondre à des problématiques de recherche en SHS. Nous utilisons un modèle de classification d'images "Zero-Shot", qui permet une grande flexibilité sans nécessiter de ré-entraînement coûteux, une approche pertinente issue de la **veille scientifique et technologique**.

### Étape 1 : Initialisation de l'Environnement

Nous commençons par importer les bibliothèques Python nécessaires et définir les chemins d'accès à notre corpus et aux fichiers de sortie. Cette étape garantit la **reproductibilité** de l'expérience.

In [ ]:
import json
from pathlib import Path
from PIL import Image
from transformers import pipeline
from IPython.display import display

# Configuration des chemins (relatifs à la racine du projet)
CORPUS_PATH = Path('../corpus/images/')
OUTPUT_JSON_PATH = Path('../corpus/ai_results.json')
MODEL_NAME = "openai/clip-vit-large-patch14"

print("Environnement configuré.")

### Étape 2 : Définition du Cadre d'Analyse Sémantique

Le choix des catégories est une étape méthodologique cruciale. Plutôt que d'utiliser des classes génériques, nous définissons ici une liste de labels spécifiquement adaptés à la nature de notre corpus (gravures techniques, photographies, etc.).

Dans un projet de recherche réel, cette **élaboration de protocole** serait menée en collaboration avec les experts du domaine (historiens, sociologues, historiens de l'art) pour s'assurer de la pertinence des axes d'analyse.

In [ ]:
# Liste de labels optimisée pour le corpus "machines"
candidate_labels = [
    "schéma technique",
    "gravure de paysage",
    "scène de travail",
    "photographie industrielle",
    "document imprimé",
    "scène historique ou allégorique",
    "illustration"
]

print("Cadre d'analyse défini avec les catégories suivantes :")
for label in candidate_labels:
    print(f"- {label}")

### Étape 3 : Instanciation du Modèle d'IA

Nous chargeons le pipeline de classification "Zero-Shot" de la bibliothèque Hugging Face. Le modèle choisi, CLIP, a été entraîné par OpenAI pour comprendre à la fois le texte et les images, ce qui lui permet de classer une image selon nos labels textuels sans avoir été spécifiquement entraîné pour cela.

In [ ]:
print("Initialisation du modèle d'IA (CLIP)... Cette opération peut prendre quelques minutes lors du premier lancement.")

# Instanciation du pipeline. Le modèle sera téléchargé si nécessaire.
classifier = pipeline(
    "zero-shot-image-classification",
    model=MODEL_NAME,
    device="cpu" # Assure la compatibilité sur toutes les machines
)

print("Modèle prêt à l'emploi.")

### Étape 4 : Exécution du Protocole d'Analyse

Le script parcourt maintenant chaque image du corpus. Pour chacune, il exécute l'inférence du modèle et stocke les résultats (la liste des labels classés par score de confiance). Le processus est transparent, affichant sa progression en temps réel. Cette approche garantit la **traçabilité** et la **reproductibilité** du traitement.

In [ ]:
image_files = sorted(list(CORPUS_PATH.glob('*.jpg')))
results = {}
total_images = len(image_files)

print(f"Début de l'analyse de {total_images} images...")

for i, image_path in enumerate(image_files):
    try:
        # Conversion en RGB pour assurer la compatibilité
        image = Image.open(image_path).convert("RGB")
        
        # Inférence du modèle
        predictions = classifier(image, candidate_labels=candidate_labels)
        
        # Stockage des résultats structurés
        results[image_path.name] = {
            'labels': [p['label'] for p in predictions],
            'scores': [round(p['score'], 4) for p in predictions]
        }
        print(f"  [{i+1}/{total_images}] Analyse de {image_path.name}... OK. (Top label: '{results[image_path.name]['labels'][0]}')")
    except Exception as e:
        print(f"    /!\\ Erreur lors de l'analyse de {image_path.name}: {e}")
        results[image_path.name] = {'error': str(e)}

print("\nAnalyse du corpus terminée.")

### Étape 5 : Sauvegarde et Valorisation des Données

L'enrichissement est terminé. Nous sauvegardons les résultats dans un fichier JSON. Ce fichier structuré est un **livrable** essentiel : il ne contient plus seulement des chemins vers des images, mais des données exploitables.

Ces données peuvent maintenant être intégrées dans des systèmes de gestion de bases de données, ou être utilisées pour générer des manifestes au format **IIIF**, ouvrant la voie à la création de **plateformes d'exploration** et de visualisation avancées.

In [ ]:
# Sauvegarde des résultats dans un fichier JSON
with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Les données enrichies ont été sauvegardées dans : {OUTPUT_JSON_PATH}")

# Affichons un exemple pour visualiser le résultat
example_image_name = '07.jpg'
example_image_path = CORPUS_PATH / example_image_name

print(f"\n--- Exemple de résultat pour l'image '{example_image_name}' ---")

# Affichage de l'image
display(Image.open(example_image_path).resize((400, 300)))

# Affichage des données JSON correspondantes
print(json.dumps(results.get(example_image_name, {}), indent=2, ensure_ascii=False))